# HEADLINE


#Data Science Technical Challenge - (Deadline: Sep 16 2024 11:59 p.m.) - Test, no es una prueba de ingreso

## Take Home: Financial Transactions


### Agosto 2024

<hr style="height:2pt">

## Descripción

El siguiente dataset contiene 116.201 registros de transacciones financieras de dominio público, enriquecida con data sintentica.

El objetivo de este ejercicio es realizar un Exploratory Data Analysis (EDA) de este dataset, para entender la información contenida y obtener insights relevantes para ciertas tareas analíticas.

Se puede descargar el dataset (formato parquet) desde este [link](https://drive.google.com/file/d/1RjeIHmtOTxz4M9WVhh8n9YsMTjk-1V80/view?usp=drive_link). Tendrás acceso a este archivo hasta la fecha límite del challenge que se te indicará en el correo.

A continuación, una descripción de las columnas:

| Variable            | Descripción                                                                 |
| :------------------ | :-------------------------------------------------------------------------- |
| account_id          | Número de cuenta involucrado en la transacción.                              |
| date                | Fecha de la transacción.                                                     |
| transaction_details | Narración o descripción de la transacción en los estados de cuenta bancarios.|
| chq_no              | Número de cheque asociado con la transacción, si corresponde.                |
| value_date          | Fecha de finalización de la transacción.                                     |
| withdrawal_amt.     | Monto retirado en la transacción.                                            |
| deposit_amt         | Monto depositado en la transacción.                                          |
| balance_amt         | Saldo actual de la cuenta después de la transacción.                         |
| category            | Categoría asignada basada en los detalles de la transacción.       |
| city                | Ciudad donde se asume que ocurrió la transacción.                  |
| device              | Tipo de dispositivo utilizado para la transacción (e.g., Móvil, Escritorio, Tablet). |


## Tareas

En este notebook se deberá cargar todas las librerías que se necesitan para explorar y procesar el dataset dado, y así realizar el analisis corresponendiente para extraer insights sobre la información dada. Se puede realizar cualquier análisis deseado, pero al final se espera encontrar realizadas las tareas del tipo "requerido". Además, hay algunos aspectos valorados del tipo "deseable" y "bonus" para obtener una valoración sobresaliente en este ejercicio.

El código debe ser desarrollado en Python >= 3.9. Los reportes pueden estar en español o inglés.

### Requerido

- **Data QA:** Se debe chequear la calidad del dataset para hacer una evaluación de qué tan apropiados son los datos para tareas de Data Science. Proponga un conjunto de correcciones en los datos de ser necesario.
- **Reporting:** Documente los resultados e insights obtenidos durante la exploración y describa conclusiones desde una perspectiva de negocio, soportado por gráficos / tablas / métricas.
- **Machine Learning:** Describa las posibles tareas de Machine Learning que podrían realizarse desde el dataset dado, que podrían ser valiosas en el dominio dado (sólo explicar, **no entrenar un modelo**).



### Deseable

- **Versionado de código con Git** (incluso puede publicarse en tu cuenta personal de GitHub!).
- **Feature Engineering:** Indicar y calcular posibles candidatos de features que podrían utilizarse tanto columnas originales y transformaciones.
- **Modelo predictivo:** Realice un modelo predictivo.
- **Mostrar skills en Python:** Teniendo buenas practicas en la estructura del código y la documentación.
- **Casos de uso:** Describir posibles casos de usos a tratar con este dataset que podrían agregar valor al negocio dado, indicando métodos / técnicas y algoritmos por cada uno de ellos, así como justificando las decisiones tomadas.
- **Métricas:** Definir y calcular las métricas que considere más relevantes para la problemática propuesta.

### Bonus

- Manejo de environment de desarrollo mediante alguna tecnología (e.g. Docker, virtualenv, conda).
- Identificar nuevos atributos / tablas que podrían ser relevantes o necesarias para un mejor análisis.

Este ejercicio está diseñado para ser completado en ~3 hs siguiendo sólo los aspectos del tipo "requerido", pero se contempla una semana para entregarlo con todos los aspectos que se deseen completar.

Una vez completado este ejercicio, por favor mandar un archivo ZIP de la
carpeta con todos los recursos usados en este trabajo (e.g. Jupyter notebook,
 scripts, documentos, imágenes, etc), también puedes compartir un collab reproducible, o bien el enlace al repositorio de
 GitHub, a `jaison.gonzalez@mercadolibre.com.co`

**Que te diviertas!**

<img src="http://s3.amazonaws.com/melidata-external/data-science-interviews/2021/img/hunger_games_data_meme.jpeg" alt="drawing" style="width:200px;"/>

### Anotaciones

- Está permitido usar las herramientas y librerias que consideres, solo ten presente que luego se te pedirá justificar.
- Aseguraté de que tu trabajo sea reproducible.

# SOLUTION

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
import logging
import os

In [2]:
sns.set_palette("pastel")

## Methods:

In [3]:
def pie(df:pd.DataFrame,column:str):
    """this plots a pie for the column specified"""
    n_counts = df[column].value_counts()
    labels = labels=n_counts.index
    plt.pie(n_counts,labels=labels, autopct='%1.1f%%', startangle=140)
    plt.title(f'Distribution of {column}')

In [4]:
def two_pies(df:pd.DataFrame,col1:str,col2:str,len1:int=4,len2:int=4):
    """it plots two concentric pies. The outter circle will correspond to the columns col1. The inner circle will correspondond to the column col2.
    The col2 will be groupped inside each group of the groups made by the col1 variable
    @len1: number of characters to take from the names in col1
    @lenw: number of characters to take from the names in col2
    """
    df = df.sort_values(by=col1)    
    df['agrupadas'] = df[col1].apply(lambda x: x.replace(" ", "")).str[:len1] + '_' + df[col2].str[:len2]

    # Make data:
    group_names = list(df[col1].unique())  # Create group names (group1 to group10)
    group_size = df[col1].value_counts().sort_index()  # Assuming all groups have equal size (adjust as needed)

    subgroup_names = list(df['agrupadas'].unique())
    subgroup_size = df['agrupadas'].value_counts().sort_index()

    # Create colors:
    num_groups = len(group_names)
    color_map = plt.get_cmap('tab10')  # Use a colormap for 10 groups
    colors = [color_map(i / num_groups) for i in range(num_groups)]

    # First Ring (outside)
    fig, ax = plt.subplots()
    ax.axis('equal')
    mypie, wedges, _ = ax.pie(group_size, radius=2.5, labels=group_names, autopct="%1.1f%%", pctdistance=0.9, colors=colors)
    plt.setp(mypie, width=0.5, edgecolor='white')

    # Second Ring (Inside)
    mypie2, _ = ax.pie(subgroup_size, radius=2.5 - 0.5, labels=subgroup_names, labeldistance=0.7, colors=colors, textprops={'fontsize': 8}, rotatelabels=True)
    plt.setp(mypie2, width=0.8, edgecolor='white')

    # Add title and legend
    plt.suptitle(f"Distribution of {col2}\nwithin {col1}", y=0.5)
    # plt.legend(mypie, group_names, loc="best")

    plt.margins(0, 0)
    plt.show()

    print(np.sort(df[col2].unique()))

In [5]:
def corr_city_month(df:pd.DataFrame, col:str):
    """
    it groups by month and city, then it computes corr in the corresponding variable col
    """

    dfg = df.groupby(["city",'month'])[[col]].sum()
    dfg = dfg.reset_index()
    dfg.head(2)

    grouped_df = dfg.groupby('city')

    matrix_df = pd.DataFrame(index=grouped_df.groups.keys(), columns=grouped_df.groups.keys())
    for group1_name, group1_data in grouped_df:
        for group2_name, group2_data in grouped_df:
            if group1_name != group2_name:
                correlation = stats.pearsonr(group1_data[col], group2_data[col])[0]
                matrix_df.loc[group1_name, group2_name] = correlation

    return matrix_df

In [6]:
def plot_hist_3d(data:pd.DataFrame, x:str, y:str, log_scale:bool=False):
    """
    it plots a 3d histogram in a 2d plane. 
    - One histogram will be computed per group in "x".
    - That histogram will show the frequencies for values in "y". 
    - The corresponding freq value will be represented by a color
    @param log_scale: True if you want that the y-values are scaled by the log function
    """
    plt.figure(figsize=(20,3))
    ax = sns.histplot(data=data,x=x,y=y, log_scale=log_scale, cmap='flare')
    plt.colorbar(ax.collections[0], label=y)
    plt.show()


## Input:

In [7]:
# load input data
workfolder = r'.\..'  # root folder
pathfile = os.path.join(workfolder,r'Data\bank_transactions.parquet')
df = pd.read_parquet(pathfile)

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
print(df.account_id.unique())

In [11]:
# remove \' to easy data management
df['account_id'] = df['account_id'].str.replace("'", "")

In [ ]:
df.isna().sum()

In [ ]:
df.describe(include='all')

In [14]:
# any register should be either withdrawal or deposit
s = df.loc[ (df.withdrawal_amt.isna()) & (df.deposit_amt.isna()) ]
if len(s):
    logging.warning(f'There is {len(s)} regs for review')
# i should have any register with values in both columns:
if (len(df) - (df.withdrawal_amt.isna().sum() + df.deposit_amt.isna().sum()))!=0:
    logging.warning('There are registers with values in both columns: withdrawal and deposit')

lets create some useful columns:

In [ ]:
df.date.unique()

In [16]:
# year of transaction
df['month'] = df['date'].apply(lambda x: x.replace(day=1))
# month of transaction
df['year'] = df['date'].apply(lambda x: x.replace(month=1, day=1))
# difference in days between end and start transaction
df['date_diff'] = (df.value_date - df.date).dt.days


In [ ]:
df[df.date_diff>0]

We have:
- a total of 116,201 transactions
- data from 10 accounts
- transactions from 2015-01-01 to 2019-03-05, i.e. 4y + 2m +4d
- 44,806 different transaction details
- 905 (0.78%) transaction were made using check
- 18 categories:
    - 'Transfer', 'Investment', 'Miscellaneous', 'Loan Payment',
    - 'Subscriptions', 'Pets & Pet Care', 'Food & Dining',
    - 'Utility Bill', 'Electronics & Gadgets', 'Insurance', 'Travel',
    - 'Shopping', 'Education', 'Health & Wellness',
    - 'Charity & Donations', 'Entertainment', 'Transportation',
    - 'Childcare & Parenting'
- 10 source cities (all in USA): 
    - 'New York',       'Phoenix',   'Dallas',  'San Jose',     'Philadelphia', 
    - 'San Antonio',    'San Diego', 'Houston', 'Los Angeles',  'Chicago'
- 3 device types: 'Tablet', 'Mobile', 'Desktop'

## See data distribution:

We can see that city and device have very nice distributions, i.e. almost equal data distribution per category:

In [ ]:
plt.figure(figsize=(8,3))
plt.subplot(121)
pie(df,'city')

plt.subplot(122)
pie(df,'device')

lets see device usage inside each city:

In [ ]:
two_pies(df,col1='city',col2='device')

lets see the category per city in the whole time:

In [ ]:
two_pies(df,col1='city',col2='category',len2=6)

In [ ]:
# withdrawals
two_pies(df[~df.withdrawal_amt.isna() ],col1='city',col2='category',len2=6)

In [ ]:
s = df[ ~(df.withdrawal_amt.isna()) ]
s = s.groupby('city').category.value_counts().reset_index().sort_values(by=['city','count'])
s
s.groupby('city').nth(-1)

In [ ]:
hasta aquí tío lafa

In [ ]:
df.loc[(df.city=='San Jose') & (df.category=='Transfer') & (~df.withdrawal_amt.isna())]

In [ ]:
# deposits
two_pies(df[(~df.deposit_amt.isna()) ],col1='city',col2='category',len2=6)

lets compare the first year 2015 vs the year 2018 ( i do not use year 2019 beacuase it does not have data from all the months)

In [ ]:
# 2015: city and device
two_pies(df[df.year=='2015-01-01'],col1='city',col2='device')

In [ ]:
# 2018: city and device
two_pies(df[df.year=='2018-01-01'],col1='city',col2='device')

In [ ]:
# 2015: city and category
two_pies(df[df.year=='2015-01-01'],col1='city',col2='category',len2=6)
print(df.category.unique())

In [ ]:
# 2018: city and category
two_pies(df[df.year=='2018-01-01'],col1='city',col2='category',len2=6)

In [ ]:
# 2015: city and category only for withdrawal
two_pies(df[(df.year=='2015-01-01') & (~df.withdrawal_amt.isna()) ],col1='city',col2='category',len2=6)

In [ ]:
# 2015: city and category only for deposits
two_pies(df[(df.year=='2015-01-01') & (~df.deposit_amt.isna()) ],col1='city',col2='category',len2=6)

Lets see withdrawals and deposits:
- the tendency (using median) is that transactions for deposits (426,500) are higher than withdrawals (47,083)
    - Thus, one may have the first impression that the market will distribute the cash in the amts and we just need to pay attetion to collect the money from amts, not placing it. We could have a further analysis
- distributions seems to be bimodal

In [ ]:
# withdrawal_amt
s = df.loc[~df.withdrawal_amt.isna(), 'withdrawal_amt']
sns.histplot(data=s, bins=50, kde=True, alpha=1, log_scale=True, label='withdrawal_amt')

# deposit_amt
s = df.loc[~df.deposit_amt.isna(), 'deposit_amt']
sns.histplot(data=s, bins=50, kde=True, alpha=0.3, log_scale=True, label='deposit')

# median values
s = df.loc[:,['withdrawal_amt','deposit_amt']].median()
sns.lineplot(x=[s.loc['withdrawal_amt'],s.loc['withdrawal_amt']], y=[0,7500], color='blue', label='median_wd')
sns.lineplot(x=[s.loc['deposit_amt'],s.loc['deposit_amt']], y=[0,7500], color='orange', label='median_dp')

plt.legend(); plt.plot(); plt.show()

In [ ]:
df.loc[:,['withdrawal_amt','deposit_amt']].describe()

some extra questions from this data:
- transctions by city
- transactions by device
    - what device is associated to the highest transaction values?
    --> all transaction are digitally made, there is no 
- transactions with bank checks, with value range manage?
- transactions by months, any yearly tendency?
- is there any category value related to the value?
- how should i distribute the cash in the city? (depending on the relation deposits vs withdrawals)
    - -> nope, i am only managing digital transaction, all transaction have a digital device associated.

We observed that all cities manage similar withdrawal transferences:

In [ ]:
plt.figure(figsize=(14,3))
ax = sns.histplot(data=df,x='city',y='withdrawal_amt', log_scale=True, cmap='flare')
plt.colorbar(ax.collections[0], label='withdrawal_amt')
plt.show()

We observed that all devices manage similar withdrawal transferences:

In [ ]:
plt.figure(figsize=(14,3))
ax = sns.histplot(data=df,x='device',y='withdrawal_amt', log_scale=True, cmap='flare')
plt.colorbar(ax.collections[0], label='withdrawal_amt')
plt.show()

We observed accounts DO NOT manage similar withdrawal transferences, nor deposit transferences:

In [ ]:
# withdrawal
plot_hist_3d(data=df,x='account_id',y='withdrawal_amt', log_scale=True)

In [ ]:
# deposit
plot_hist_3d(data=df,x='account_id',y='deposit_amt', log_scale=True)

In [ ]:
# balance
plot_hist_3d(data=df,x='account_id',y='balance_amt', log_scale=False)

In [ ]:
# withdrawals
plot_hist_3d(data=df[~df.withdrawal_amt.isna()],x='account_id',y='category', log_scale=False)

In [ ]:
# deposits
plot_hist_3d(data=df[~df.deposit_amt.isna()], x='account_id', y='category', log_scale=False)

In [ ]:
# cities. only withdrawals
plot_hist_3d(data=df[~df.withdrawal_amt.isna()], x='account_id', y='city', log_scale=False)

In [ ]:
# cities. only deposit
plot_hist_3d(data=df[~df.deposit_amt.isna()], x='account_id', y='city', log_scale=False)

## Bank checks analysis:
- Checks are used only for withdrawals, no one for deposit

In [ ]:
print(df.loc[~df.chq_no.isna(), 'deposit_amt'].describe())
print(df.loc[~df.chq_no.isna(), 'withdrawal_amt'].describe())

In [ ]:
plt.figure(figsize=(3,3))
sns.histplot(df.loc[~df.chq_no.isna(), 'withdrawal_amt'], log_scale=True)
plt.show()

## by months:

In [40]:
df = df.sort_values(by='date')

- general by month

In [ ]:
s = df.groupby('month')[['withdrawal_amt','deposit_amt']].sum().reset_index()
sns.lineplot(s,x='month',y='withdrawal_amt', label='withdrawal_amt')
sns.lineplot(s,x='month',y='deposit_amt', label='deposit_amt')
plt.legend(); plt.show()

- by month and city:

In [ ]:
s = df.groupby(['city','month'])[['withdrawal_amt','deposit_amt']].sum().reset_index()
plt.figure(figsize=(12,4))
plt.subplot(121)
sns.lineplot(data=s,x='month',y='withdrawal_amt', hue='city')
plt.title('withdrawal_amt')

plt.subplot(122)
sns.lineplot(data=s,x='month',y='deposit_amt', hue='city')
plt.title('deposit_amt')

plt.show()

## Lets see correlation between information per city

In [ ]:
# group per city and month, and compute correlation for withdrawals
s = corr_city_month(df, col='withdrawal_amt')
display(s)
print(s.min().min())

In [ ]:
s = corr_city_month(df, col='deposit_amt')
display(s)
print(s.min().min())

Per city, data series are very similar. All have a corr>84% for withdrawals and deposits

Thus, lets analyze the accounts 

## balance

we can see that the bank balance is always negative and it is in average decreasing at a speed of
-23,865,986,426 per month!

In [45]:
def linear_tendency(s,col:str):
    """it computes linear tendency to series input over the column col"""
    x = np.arange(s.shape[0])
    linear_fit = np.polyfit(x, s[col], 1)
    linear_trend = np.poly1d(linear_fit)
    return linear_fit, linear_trend(x)

In [ ]:
# group balance by month
s = df.groupby('month').balance_amt.sum()
s = s.reset_index()
print(s.balance_amt.describe())

# plot behaviour
sns.lineplot(data=s,x=s.month,y=s.balance_amt, marker='o')

# see the linear tendency of general balance
linear_fit, linear_trend = linear_tendency(s,'balance_amt')
plt.plot(s.month.to_list(), linear_trend, linestyle='--', color='red')

plt.title('Balance per month')
plt.show()

# print the slope, 
print('Slope: ', linear_fit[0])

- balance per city

behaviour in all cities have the same behaviour, debts from users to bank have a strong increasement in july 2015 and decreases in march 2019, where the general balance returns to similar values as at the beggining

In [ ]:
# group balance by city and month
s = df.groupby(['city', 'year','month']).balance_amt.sum()
s = s.reset_index()
s
# # plot behaviour
for y in s.year.unique():
    sy = s.loc[s.year==y,['city','month','balance_amt']]
    plt.figure(figsize=(12,3))
    sns.barplot(data=sy, x='month', y='balance_amt', hue='city')
    plt.title(f'Balance per month. Year {y}')
    plt.show()

lets compare the first three months from 2015 and 2019:

In [ ]:
s = df[(df.month<='2015-06-01') | (df.month>='2019-01-01')]
plt.figure(figsize=(12,3))
sns.barplot(data=s, x='month', y='balance_amt', hue='city')
plt.title(f'Balance per month. Year {y}')
plt.show()

## lest see one account

In [ ]:
df.groupby('account_id').date.count().sort_values()

lets work with the accout 1196428 due to it has more data

- balance per account

In [50]:
dfq = df[df.account_id=='409000493201'].reset_index(drop=True)

In [ ]:
# group balance by city and month
s = dfq.groupby(['city', 'year','month']).balance_amt.sum()
s = s.reset_index()
s
# # plot behaviour
for y in s.year.unique():
    sy = s.loc[s.year==y,['city','month','balance_amt']]
    plt.figure(figsize=(12,3))
    sns.barplot(data=sy, x='month', y='balance_amt', hue='city')
    plt.title(f'Balance per month. Year {y}')
    plt.show()

In [ ]:
# withdrawal transactions
two_pies(dfq[(dfq.year=='2016-01-01') & (~dfq.withdrawal_amt.isna()) ],col1='city',col2='category',len2=6)

In [ ]:
# desposit transactions
two_pies(dfq[(dfq.year=='2018-01-01') & (~dfq.deposit_amt.isna()) ],col1='city',col2='category',len2=6)